# Label Creation
In the case of this deep-learning examination of Hail-Event data, the events themselves will be converted to labels of the form of gray-scale objects of values 0-1 on a [21 x 21] matrix, one for each possible hour accept for the first 2 hours of Jan 1, 2010 and the last 2 hours of December 31, 2022.

The data itself comes from NOAA weather-event records. These records must be collected in parts as the website itself cannot display more than 500 at a time. After extracting data from the states of Indiana, Ohio, Illinois, Michigan, and Kentucky, the individual files for each state where concatenated together in excel to create 5 record-files, one for each state, contained in the file **[Bulk_Weather_Events]**. This data was edited as the concatenation process and the poorness of the record keeping itself lead ot problems with the data itself, including severalfaulty instances or faulty comma-serpation for the original csv files. What records that were unsalvagable had to be removed, several others had to be edited by hand into their proper intended form.

After this, the data was then edited to remove any instances where the date or time for both begining and end where not known. Where only one was unknown, in was turned into a short as possible instance event where the missing value for the begin/end date-time was filled in by the known begin/end date time. That is, if the end-time of a hail-event was known, but not the begining, then the begining was filled in with the end-value and considered a temporary 1-hour instance.

After this, we then removed all instances which did not end after Jan 1, 2010 or began after December 31, 2022. We then removed any instances that did not begin nor end in the range of (37,42) Latitude and (-89,-84) Longitude. We then converted values which began or ended outside that boundry to begin/end at the closest boundry to ensure all instances where within the bounded study area.

In extracting the datetime of the event, we round down to the nearest hour, so an event which began at 7:15 and went till 9:25 will instead have began at 7:00 and ended at 9:00. However, when stored, the event will include times (7=>8), (8=>9), and (9=>10). While this might lead to artificial inflating of the instances, it is close enough to consider safe for the dataset.

## Next step
Once we have cleaned the data instances, we must make a blank 21 by 21 array for 113948 instances, one for each hour between the 2nd of Jan 1, 2010 to the 21st of December 31, 2022, including both ends of these hours. We then must convert the begin and end latitudes into a fitting of this range. Any which traveled a certain latitude will be considered having been a larger than average area during the entire start to end period, creating a rectangular-area of hail-events during that time-period. Any which began in one of the 21-latitude/longitude ranges and traveled into another will be considered a part of both ranges. So if a hail event began in Longitude -88.65 and ended in Longitude -88.45, then we will count in both in the cell representing range (-88.75 => -88.5) and (-88.5 => -88.25).

In [ ]:
from google.colab import drive
#drive.mount('/content/gdrive')
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import datetime
import os
import sklearn.preprocessing as skp
import collections
import time

In [ ]:
print(os.getcwd())
os.chdir('gdrive/MyDrive/Deep_Learning_Project')
print(os.getcwd())

/content
/content/gdrive/MyDrive/Deep_Learning_Project


In [ ]:
x = []
for filename in os.listdir('Bulk_Weather_Events'):
    x.append(pd.read_excel('Bulk_Weather_Events/' + filename))
x = pd.concat(x, axis = 0, ignore_index=True)

x.shape

(28576, 43)

In [ ]:
x.loc[x['BEGIN_LAT']==' ', :] = np.nan
x.loc[x['BEGIN_LON']==' ', :] = np.nan
x.loc[x['BEGIN_TIME']==' ', :] = np.nan
x.loc[x['BEGIN_DATE']==' ', :] = np.nan
x.loc[x['END_LAT']==' ', :] = np.nan
x.loc[x['END_LON']==' ', :] = np.nan
x.loc[x['END_TIME']==' ', :] = np.nan
x.loc[x['END_DATE']==' ', :] = np.nan

In [ ]:
x = x[['BEGIN_DATE', 'END_DATE', 'BEGIN_TIME', 'END_TIME', 'BEGIN_LAT', 'END_LAT', 'BEGIN_LON', 'END_LON']]
x.isnull().sum()

BEGIN_DATE    191
END_DATE      200
BEGIN_TIME    170
END_TIME      191
BEGIN_LAT     259
END_LAT       259
BEGIN_LON     259
END_LON       259
dtype: int64

In [ ]:
x = x[~((x['BEGIN_DATE'].isnull()) & (x['END_DATE'].isnull()))]
x = x[~((x['BEGIN_TIME'].isnull()) & (x['END_TIME'].isnull()))]
x = x[~((x['BEGIN_LAT'].isnull()) & (x['END_LAT'].isnull()) & (x['BEGIN_LON'].isnull()) & (x['END_LON'].isnull()))]
x.isnull().sum()

BEGIN_DATE    0
END_DATE      9
BEGIN_TIME    0
END_TIME      0
BEGIN_LAT     0
END_LAT       0
BEGIN_LON     0
END_LON       0
dtype: int64

In [ ]:
x['END_DATE'] = x.apply(lambda y: y['BEGIN_DATE'] if pd.isnull(y['END_DATE']) else y['END_DATE'], axis = 1)
x['END_TIME'] = x.apply(lambda y: y['BEGIN_TIME'] if pd.isnull(y['END_TIME']) else y['END_TIME'], axis = 1)

x['END_LAT'] = x.apply(lambda y: y['BEGIN_LAT'] if pd.isnull(y['END_LAT']) else y['END_LAT'], axis = 1)
x['BEGIN_LAT'] = x.apply(lambda y: y['END_LAT'] if pd.isnull(y['BEGIN_LAT']) else y['BEGIN_LAT'], axis = 1)
x['END_LON'] = x.apply(lambda y: y['BEGIN_LON'] if pd.isnull(y['END_LON']) else y['END_LON'], axis = 1)
x['BEGIN_LON'] = x.apply(lambda y: y['END_LON'] if pd.isnull(y['BEGIN_LON']) else y['BEGIN_LON'], axis = 1)

In [ ]:
x[['BEGIN_LAT', 'END_LAT', 'BEGIN_LON', 'END_LON']] = x[['BEGIN_LAT', 'END_LAT', 'BEGIN_LON', 'END_LON']].apply(pd.to_numeric)

In [ ]:
def int_to_time(weird):
    minute = weird % 100
    hour = weird // 100
    if minute < 10:
        minute = f"0{int(minute)}"
    else:
        minute = f"{int(minute)}"
    if hour < 10:
        hour = f"0{int(hour)}"
    else:
        hour = f"{int(hour)}"
    our_time =  f"{hour}"
    return datetime.datetime.strptime(our_time, '%H').time()


x['BEGIN_DT'] = x.apply(lambda y: datetime.datetime.combine(y['BEGIN_DATE'], int_to_time(y['BEGIN_TIME'])), axis = 1)
x['END_DT'] = x.apply(lambda y: datetime.datetime.combine(y['END_DATE'], int_to_time(y['END_TIME'])), axis = 1)

x.dtypes

BEGIN_DATE    datetime64[ns]
END_DATE      datetime64[ns]
BEGIN_TIME           float64
END_TIME             float64
BEGIN_LAT            float64
END_LAT              float64
BEGIN_LON            float64
END_LON              float64
BEGIN_DT      datetime64[ns]
END_DT        datetime64[ns]
dtype: object

In [ ]:
x = x[x['END_DT'] >= "2011-01-01 02:00:00"]
x = x[x['BEGIN_DT'] <= "2022-12-31 21:00:00"]
print(x.shape)

(13055, 10)


In [ ]:
x = x[(x['BEGIN_LAT'].between(37, 42) | x['END_LAT'].between(37, 42)) &
  (x['BEGIN_LON'].between(-89, -84) | x['END_LON'].between(-89, -84))]

print(x.shape)

(5900, 10)


In [ ]:
print(x['BEGIN_LAT'][x['BEGIN_LAT'] < 37].shape)
print(x['BEGIN_LAT'][x['BEGIN_LAT'] > 42].shape)
print(x['END_LAT'][x['END_LAT'] < 37].shape)
print(x['END_LAT'][x['END_LAT'] > 42].shape)

print(x['BEGIN_LON'][x['BEGIN_LON'] < -89].shape)
print(x['BEGIN_LON'][x['BEGIN_LON'] > -84].shape)
print(x['END_LON'][x['END_LON'] < -89].shape)
print(x['END_LON'][x['END_LON'] > -84].shape, end='\n\n')

#Now reset the values:
x['BEGIN_LAT'][x['BEGIN_LAT'] < 37] = 37
x['BEGIN_LAT'][x['BEGIN_LAT'] > 42] = 41.99
x['END_LAT'][x['END_LAT'] < 37] = 37
x['END_LAT'][x['END_LAT'] > 42] = 41.99
x['BEGIN_LON'][x['BEGIN_LON'] < -89] = -88.99
x['BEGIN_LON'][x['BEGIN_LON'] > -84] = -84
x['END_LON'][x['END_LON'] < -89] = -88.99
x['END_LON'][x['END_LON'] > -84] = -84

print(x['BEGIN_LAT'][x['BEGIN_LAT'] < 37].shape)
print(x['BEGIN_LAT'][x['BEGIN_LAT'] > 42].shape)
print(x['END_LAT'][x['END_LAT'] < 37].shape)
print(x['END_LAT'][x['END_LAT'] > 42].shape)

print(x['BEGIN_LON'][x['BEGIN_LON'] < -89].shape)
print(x['BEGIN_LON'][x['BEGIN_LON'] > -84].shape)
print(x['END_LON'][x['END_LON'] < -89].shape)
print(x['END_LON'][x['END_LON'] > -84].shape, end='\n\n')

(2,)
(0,)
(3,)
(2,)
(7,)
(0,)
(0,)
(3,)

(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)



<ipython-input-13-000db80959c5>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['BEGIN_LAT'][x['BEGIN_LAT'] > 42] = 41.99
<ipython-input-13-000db80959c5>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['END_LAT'][x['END_LAT'] > 42] = 41.99
<ipython-input-13-000db80959c5>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['BEGIN_LON'][x['BEGIN_LON'] > -84] = -84
<ipython-input-13-000db80959c5>:19: SettingWithCopyWarning: 
A val

In [ ]:
x = x.reset_index()

In [ ]:
x['Steps_per_Event'] = x.apply(lambda d: pd.date_range(d['BEGIN_DT'], d['END_DT'], freq='h'),axis = 1)
#df.explode('Start Date')
x = x.explode('Steps_per_Event')
#Only a few duplicates

#Count the indexes as needed.
#Turn into hourly distance from time
x['Steps_per_Event'].min()
print(x['Steps_per_Event'].map(lambda y: y.year).max())
print(x['Steps_per_Event'].map(lambda y: y.year).min())

x

2022
2011


,BEGIN_DATE,END_DATE,BEGIN_TIME,END_TIME,BEGIN_LAT,END_LAT,BEGIN_LON,END_LON,BEGIN_DT,END_DT,Steps_per_Event
1584,2011-02-27,2011-02-27,2052.0,2053.0,40.4800,40.4800,-88.9800,-88.9800,2011-02-27 20:00:00,2011-02-27 20:00:00,2011-02-27 20:00:00
1585,2011-02-27,2011-02-27,2056.0,2057.0,40.5345,40.5345,-89.0000,-89.0000,2011-02-27 20:00:00,2011-02-27 20:00:00,2011-02-27 20:00:00
1586,2011-02-27,2011-02-27,2100.0,2101.0,40.5507,40.5507,-88.9596,-88.9596,2011-02-27 21:00:00,2011-02-27 21:00:00,2011-02-27 21:00:00
1587,2011-02-27,2011-02-27,2103.0,2104.0,40.4800,40.4800,-88.9800,-88.9800,2011-02-27 21:00:00,2011-02-27 21:00:00,2011-02-27 21:00:00
1588,2011-02-27,2011-02-27,2122.0,2123.0,39.4366,39.4366,-88.3700,-88.3700,2011-02-27 21:00:00,2011-02-27 21:00:00,2011-02-27 21:00:00
...,...,...,...,...,...,...,...,...,...,...,...
25848,2022-06-13,2022-06-13,1628.0,1630.0,39.4400,39.4400,-84.5400,-84.5400,2022-06-13 16:00:00,2022-06-13 16:00:00,2022-06-13 16:00:00
25850,2022-06-13,2022-06-13,2234.0,2236.0,40.4900,40.4900,-84.3200,-84.3200,2022-06-13 22:00:00,2022-06-13 22:00:00,2022-06-13 22:00:00
25873,2022-07-05,2022-07-05,1222.0,1224.0,39.2300,39.2300,-84.2900,-84.2900,2022-07-05 12:00:00,2022-07-05 12:00:00,2022-07-05 12:00:00
25874,2022-07-05,2022-07-05,1222.0,1224.0,39.2400,39.2400,-84.2900,-84.2900,2022-07-05 12:00:00,2022-07-05 12:00:00,2022-07-05 12:00:00


In [ ]:
x['BEGIN_LAT'] = abs(pd.cut(x['BEGIN_LAT'], bins = 20, right = False, labels = False) - 19)
x['END_LAT'] = abs(pd.cut(x['END_LAT'], bins = 20, right = False, labels = False) - 19)
x['BEGIN_LON'] = pd.cut(x['BEGIN_LON'], bins = 20, labels = False)
x['END_LON'] = pd.cut(x['END_LON'], bins = 20, labels = False)

print(pd.unique(x['BEGIN_LAT']))
print(pd.unique(x['BEGIN_LON']))
print(pd.unique(x['END_LAT']))
print(pd.unique(x['END_LON']))

[ 6  5 10 14  0  2 11 18 16 17 19 15 12  8  7  1 13  4  9  3]
[ 0  2  3  1  4  5  7  8 14  9 10 16 13 12 15 11  6 17 19 18]
[ 6  5 10 14  0  2 11 18 17 16 19 15 12  8  7  1 13  4  9  3]
[ 0  2  3  1  4  5  7  8 14  9 10 16 13 12 15 11  6 17 19 18]


In [ ]:
x

,BEGIN_DATE,END_DATE,BEGIN_TIME,END_TIME,BEGIN_LAT,END_LAT,BEGIN_LON,END_LON,BEGIN_DT,END_DT,Steps_per_Event
1584,2011-02-27,2011-02-27,2052.0,2053.0,6,6,0,0,2011-02-27 20:00:00,2011-02-27 20:00:00,2011-02-27 20:00:00
1585,2011-02-27,2011-02-27,2056.0,2057.0,5,5,0,0,2011-02-27 20:00:00,2011-02-27 20:00:00,2011-02-27 20:00:00
1586,2011-02-27,2011-02-27,2100.0,2101.0,5,5,0,0,2011-02-27 21:00:00,2011-02-27 21:00:00,2011-02-27 21:00:00
1587,2011-02-27,2011-02-27,2103.0,2104.0,6,6,0,0,2011-02-27 21:00:00,2011-02-27 21:00:00,2011-02-27 21:00:00
1588,2011-02-27,2011-02-27,2122.0,2123.0,10,10,2,2,2011-02-27 21:00:00,2011-02-27 21:00:00,2011-02-27 21:00:00
...,...,...,...,...,...,...,...,...,...,...,...
25848,2022-06-13,2022-06-13,1628.0,1630.0,10,10,17,17,2022-06-13 16:00:00,2022-06-13 16:00:00,2022-06-13 16:00:00
25850,2022-06-13,2022-06-13,2234.0,2236.0,6,6,18,18,2022-06-13 22:00:00,2022-06-13 22:00:00,2022-06-13 22:00:00
25873,2022-07-05,2022-07-05,1222.0,1224.0,11,11,18,18,2022-07-05 12:00:00,2022-07-05 12:00:00,2022-07-05 12:00:00
25874,2022-07-05,2022-07-05,1222.0,1224.0,11,11,18,18,2022-07-05 12:00:00,2022-07-05 12:00:00,2022-07-05 12:00:00


In [ ]:
print(abs(x['END_LAT'] - x['BEGIN_LAT']).max())
print(abs(x['END_LON'] - x['BEGIN_LON']).max())

print(abs(x['END_LAT'] - x['BEGIN_LAT']).min())
print(abs(x['END_LON'] - x['BEGIN_LON']).min())

1
1
0
0


In [ ]:
x['Steps_per_Event'] = x['Steps_per_Event'].map(lambda x:
                                                int((x - datetime.datetime(2011, 1, 1, 2, 0, 0)).total_seconds()/3600))
x

,BEGIN_DATE,END_DATE,BEGIN_TIME,END_TIME,BEGIN_LAT,END_LAT,BEGIN_LON,END_LON,BEGIN_DT,END_DT,Steps_per_Event
1584,2011-02-27,2011-02-27,2052.0,2053.0,6,6,0,0,2011-02-27 20:00:00,2011-02-27 20:00:00,1386
1585,2011-02-27,2011-02-27,2056.0,2057.0,5,5,0,0,2011-02-27 20:00:00,2011-02-27 20:00:00,1386
1586,2011-02-27,2011-02-27,2100.0,2101.0,5,5,0,0,2011-02-27 21:00:00,2011-02-27 21:00:00,1387
1587,2011-02-27,2011-02-27,2103.0,2104.0,6,6,0,0,2011-02-27 21:00:00,2011-02-27 21:00:00,1387
1588,2011-02-27,2011-02-27,2122.0,2123.0,10,10,2,2,2011-02-27 21:00:00,2011-02-27 21:00:00,1387
...,...,...,...,...,...,...,...,...,...,...,...
25848,2022-06-13,2022-06-13,1628.0,1630.0,10,10,17,17,2022-06-13 16:00:00,2022-06-13 16:00:00,100358
25850,2022-06-13,2022-06-13,2234.0,2236.0,6,6,18,18,2022-06-13 22:00:00,2022-06-13 22:00:00,100364
25873,2022-07-05,2022-07-05,1222.0,1224.0,11,11,18,18,2022-07-05 12:00:00,2022-07-05 12:00:00,100882
25874,2022-07-05,2022-07-05,1222.0,1224.0,11,11,18,18,2022-07-05 12:00:00,2022-07-05 12:00:00,100882


In [ ]:
print(x['Steps_per_Event'].max() + (32*24))
print((365+366+365+365+365+366+365+365+365+366+365+365)*24 - 4)
#Works
x = x.reset_index()
x

105188
105188


,index,BEGIN_DATE,END_DATE,BEGIN_TIME,END_TIME,BEGIN_LAT,END_LAT,BEGIN_LON,END_LON,BEGIN_DT,END_DT,Steps_per_Event
0,1584,2011-02-27,2011-02-27,2052.0,2053.0,6,6,0,0,2011-02-27 20:00:00,2011-02-27 20:00:00,1386
1,1585,2011-02-27,2011-02-27,2056.0,2057.0,5,5,0,0,2011-02-27 20:00:00,2011-02-27 20:00:00,1386
2,1586,2011-02-27,2011-02-27,2100.0,2101.0,5,5,0,0,2011-02-27 21:00:00,2011-02-27 21:00:00,1387
3,1587,2011-02-27,2011-02-27,2103.0,2104.0,6,6,0,0,2011-02-27 21:00:00,2011-02-27 21:00:00,1387
4,1588,2011-02-27,2011-02-27,2122.0,2123.0,10,10,2,2,2011-02-27 21:00:00,2011-02-27 21:00:00,1387
...,...,...,...,...,...,...,...,...,...,...,...,...
6013,25848,2022-06-13,2022-06-13,1628.0,1630.0,10,10,17,17,2022-06-13 16:00:00,2022-06-13 16:00:00,100358
6014,25850,2022-06-13,2022-06-13,2234.0,2236.0,6,6,18,18,2022-06-13 22:00:00,2022-06-13 22:00:00,100364
6015,25873,2022-07-05,2022-07-05,1222.0,1224.0,11,11,18,18,2022-07-05 12:00:00,2022-07-05 12:00:00,100882
6016,25874,2022-07-05,2022-07-05,1222.0,1224.0,11,11,18,18,2022-07-05 12:00:00,2022-07-05 12:00:00,100882


In [ ]:
label_area = np.zeros((105188, 20, 20, 1))
label_area.sum()

0.0

In [ ]:
j = 0
for r,i in x.iterrows():
    label_area[i['Steps_per_Event'], i['BEGIN_LAT']:i['END_LAT'] + 1, i['BEGIN_LON']:i['END_LON'] + 1, 0] = 1
    #Ah, does it for columns
label_area.sum()

4185.0

In [ ]:
for filename in sorted(os.listdir('ERA_Numpy_Files')):
  print(('ERA_Numpy_Files/'+filename))

ERA_Numpy_Files/timeseries_array_#001.npy
ERA_Numpy_Files/timeseries_array_#002.npy
ERA_Numpy_Files/timeseries_array_#003.npy
ERA_Numpy_Files/timeseries_array_#004.npy
ERA_Numpy_Files/timeseries_array_#005.npy
ERA_Numpy_Files/timeseries_array_#006.npy
ERA_Numpy_Files/timeseries_array_#007.npy
ERA_Numpy_Files/timeseries_array_#008.npy
ERA_Numpy_Files/timeseries_array_#009.npy
ERA_Numpy_Files/timeseries_array_#010.npy
ERA_Numpy_Files/timeseries_array_#011.npy
ERA_Numpy_Files/timeseries_array_#012.npy
ERA_Numpy_Files/timeseries_array_#013.npy
ERA_Numpy_Files/timeseries_array_#014.npy
ERA_Numpy_Files/timeseries_array_#015.npy
ERA_Numpy_Files/timeseries_array_#016.npy
ERA_Numpy_Files/timeseries_array_#017.npy
ERA_Numpy_Files/timeseries_array_#018.npy
ERA_Numpy_Files/timeseries_array_#019.npy
ERA_Numpy_Files/timeseries_array_#020.npy
ERA_Numpy_Files/timeseries_array_#021.npy
ERA_Numpy_Files/timeseries_array_#022.npy
ERA_Numpy_Files/timeseries_array_#023.npy
ERA_Numpy_Files/timeseries_array_#

In [ ]:
number_examples = []
starter = time.time()
#Create a test of the ability to divide examples into files to better fit into a loader alongside the mounth files for features.
for filename in os.listdir('ERA_Numpy_Files'):
  open_test = np.load('ERA_Numpy_Files/'+filename, mmap_mode='r')
  number_examples.append(open_test.shape[0])
print(time.time() - starter)
cumulative_examples = np.array(number_examples).cumsum()
print(cumulative_examples)

128.65498065948486
[   744   1416   2160   2880   3624   4344   5088   5832   6552   7296
   8016   8760   9504  10200  10944  11664  12408  13128  13872  14616
  15336  16080  16800  17544  18288  18960  19704  20424  21168  21888
  22632  23376  24096  24840  25560  26304  27048  27720  28464  29184
  29928  30648  31392  32136  32856  33600  34320  35064  35808  36480
  37224  37944  38688  39408  40152  40896  41616  42360  43080  43824
  44568  45264  46008  46728  47472  48192  48936  49680  50400  51144
  51864  52608  53352  54024  54768  55488  56232  56952  57696  58440
  59160  59904  60624  61368  62112  62784  63528  64248  64992  65712
  66456  67200  67920  68664  69384  70128  70872  71544  72288  73008
  73752  74472  75216  75960  76680  77424  78144  78888  79632  80328
  81072  81792  82536  83256  84000  84744  85464  86208  86928  87672
  88416  89088  89832  90552  91296  92016  92760  93504  94224  94968
  95688  96432  97176  97848  98592  99312 100056 100776 1

In [ ]:
number_examples

[744,
 672,
 744,
 720,
 744,
 720,
 744,
 744,
 720,
 744,
 720,
 744,
 744,
 696,
 744,
 720,
 744,
 720,
 744,
 744,
 720,
 744,
 720,
 744,
 744,
 672,
 744,
 720,
 744,
 720,
 744,
 744,
 720,
 744,
 720,
 744,
 744,
 672,
 744,
 720,
 744,
 720,
 744,
 744,
 720,
 744,
 720,
 744,
 744,
 672,
 744,
 720,
 744,
 720,
 744,
 744,
 720,
 744,
 720,
 744,
 744,
 696,
 744,
 720,
 744,
 720,
 744,
 744,
 720,
 744,
 720,
 744,
 744,
 672,
 744,
 720,
 744,
 720,
 744,
 744,
 720,
 744,
 720,
 744,
 744,
 672,
 744,
 720,
 744,
 720,
 744,
 744,
 720,
 744,
 720,
 744,
 744,
 672,
 744,
 720,
 744,
 720,
 744,
 744,
 720,
 744,
 720,
 744,
 744,
 696,
 744,
 720,
 744,
 720,
 744,
 744,
 720,
 744,
 720,
 744,
 744,
 672,
 744,
 720,
 744,
 720,
 744,
 744,
 720,
 744,
 720,
 744,
 744,
 672,
 744,
 720,
 744,
 720,
 744,
 744,
 720,
 744,
 1460]

In [ ]:
cumulative_examples = list(cumulative_examples)
cumulative_examples.insert(0, 0)
print(len(cumulative_examples))

144


In [ ]:
label_area[number_examples[0]:number_examples[1]].shape

for i in range(1, len(cumulative_examples)):
  #print(i)
  z = label_area[cumulative_examples[i-1]:cumulative_examples[i]]
  if i < 10:
    i = f"00{i}"
  elif i < 100:
    i = f"0{i}"
  np.save(f'NOAA_Event_Labels/label_array_#{i}', z)

In [ ]:
z.shape

(1460, 20, 20, 1)

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)
if len(gpus)>0:
    print("GPU is available.")
else:
    print("GPU is not available. Training on CPU.")

Name: /physical_device:GPU:0   Type: GPU
GPU is available.


In [ ]:
number_examples[i]

105188

In [ ]:
x.groupby(['Steps_per_Event','BEGIN_LAT', 'BEGIN_LON']).size().reset_index().rename(columns={0:'count'})
#Clearly, we have some overlap, even with begin_lon. So not a great proposition.

,Steps_per_Event,BEGIN_LAT,BEGIN_LON,count
0,1386,5,0,1
1,1386,6,0,1
2,1387,5,0,1
3,1387,6,0,1
4,1387,10,2,1
...,...,...,...,...
4173,104418,17,13,1
4174,104419,15,16,1
4175,104419,15,17,2
4176,104419,15,18,2


In [ ]:
(datetime.datetime(2022, 12, 31, 21, 0, 0) - datetime.datetime(2010, 1, 1, 2, 0, 0)).total_seconds()/3600

113947.0